# Electons in metal I: Drude theory

This notebook is to accompany the _Electrons in metal I_ content.

Version 1.0, updated 19/08/2021 by AJM

## Import packages

To streamline operations in Python, packages can be imported to perform a host of various tasks. To make this process as simple as possible, all the required packages are included in the file _[SSP.py](https://github.com/Andy-UTAS/Solid-state/blob/master/SSP.py)_ and thus we can import all of the content: 

In [ ]:
from SSP import *

# Plot scattering of electrons in external field

In [ ]:
# Use colors from the default color cycle
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
blue, red = default_colors[0], default_colors[3]

walkers = 10 # number of particles
tau = 1 # relaxation time
gamma = .3 # dissipation strength
a = 1 # acceleration
dt = .05 # infinitesimal
T = 25 # simulation time

v = np.zeros((2, int(T // dt), walkers), dtype=float) #

# Select random time steps and scattering angles
np.random.seed(1)
scattering_events = np.random.binomial(1, dt/tau, size=v.shape[1:])
angles = np.random.uniform(high=2*np.pi, size=scattering_events.shape) * scattering_events
rotations = np.array(
    [[np.cos(angles), np.sin(angles)],
     [-np.sin(angles), np.cos(angles)]]
)

for step in range(1, v.shape[1]):
    v[:, step] = v[:, step-1]
    v[0, step] += a * dt
    v[:, step] = np.einsum(
        'ijk,jk->ik',
        rotations[:, :, step-1, :],
        v[:, step, :]
    ) * (1 - gamma * scattering_events[step-1])

r = np.cumsum(v * dt, axis=1)

scattering_positions = np.copy(r)
scattering_positions[:, ~scattering_events.astype(bool)] = np.nan

scatter_pts = scattering_positions

r_min = np.min(r.reshape(2, -1), axis=1) - 1
r_max = np.max(r.reshape(2, -1), axis=1) + 1

fig = plt.figure(figsize=(9, 6))
ax = fig.add_subplot(1, 1, 1)
ax.axis("off")
ax.set(xlim=(r_min[0], r_max[0]), ylim=(r_min[1], r_max[1]))

trajectories = ax.plot([],[], lw=1, color=blue, alpha=0.5)[0]
scatterers = ax.scatter([], [], s=20, c=red)

def frame(i):
    concatenated_lines = np.concatenate(
        (r[:, :i], np.nan * np.ones((2, 1, walkers))),
        axis=1
    ).transpose(0, 2, 1).reshape(2, -1)
    trajectories.set_data(*concatenated_lines)
    scatterers.set_offsets(scatter_pts[:, :i].reshape(2, -1).T)

anim = animation.FuncAnimation(fig, frame, frames = len(v[0]), interval= 1/60 * 1000)

plt.close()

# with open("03_scattering.html", "w") as f:
#     print(anim.to_html5_video(), file=f)
    
HTML(anim.to_html5_video())